In [ ]:
from glob import glob
import os
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
import matplotlib.ticker as ticker
from matplotlib.collections import Collection, LineCollection
from matplotlib.artist import allow_rasterization
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.dates as md
import matplotlib.ticker as ticker
from metpy.plots import ctables
#from pyPIPS import disdrometer_module as dis
from pyPIPS import pips_io as pipsio
from pyPIPS import timemodule as tm
from pyPIPS.utils import mtokm, getTimeWindow, interp_along_1D
from pyPIPS import plotmodule as pm
from pyPIPS import PIPS as pips
from pyPIPS import fmcw
from itertools import cycle
import netCDF4 as netcdf
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from skimage.restoration import unwrap_phase
import skimage.feature as SF
import numpy.ma as ma
from scipy import ndimage, interpolate
%load_ext autoreload
%autoreload 2

In [ ]:
already_merged=True

# fmcw_data_base_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2017/FMCW'
# fmcw_data_base_dir = '/Users/dawson29/sshfs_mounts/depot_robin/share/VORTEX-SE/2017/data/radar/umassfmcw/nc'
fmcw_data_base_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/2017/FMCW'

# Uncomment below for March 27th, 2017 case
date = '0327'
starttimestring = '201703271900'
endtimestring = '201703272200'

datetime_start = datetime.strptime(starttimestring, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(endtimestring, '%Y%m%d%H%M%S')

starttimestring2 = datetime_start.strftime('%Y-%m-%dT%H:%M')
endtimestring2 = datetime_end.strftime('%Y-%m-%dT%H:%M')

# Uncomment below for April 30th, 2017 case
# date = '0430'
# starttimestring = '201704302000'
# endtimestring = '201704302359

# fmcw_data_dir = os.path.join(fmcw_data_base_dir, date)
fmcw_data_dir = os.path.join(fmcw_data_base_dir, date)
fmcw_output_dir = os.path.join(fmcw_output_base_dir, date)
if not os.path.exists(fmcw_output_dir):
        os.makedirs(fmcw_output_dir)

if not already_merged:
    filepaths = fmcw.get_fmcw_filepaths(fmcw_data_dir, starttimestring, endtimestring, subdirs=False)
    print(filepaths)

    # Read in multifile FMCW dataset
    fmcw_merged_dataset = fmcw.read_fmcw_multifile_xarray(filepaths)
    # Clip to chosen times
    fmcw_merged_dataset = fmcw_merged_dataset.sel(time=slice(starttimestring2, endtimestring2))
    # Dealias
    fmcw_merged_dataset = fmcw.dealias(fmcw_merged_dataset)
    fmcw_merged_dataset.load()
    
else:    
    fmcw_filename = 'fmcw_merged_corrected_{}_{}.nc'.format(starttimestring, endtimestring)
    fmcw_filepath = os.path.join(fmcw_data_dir, fmcw_filename)
    fmcw_merged_dataset = xr.open_dataset(fmcw_filepath)

print(fmcw_merged_dataset)

In [ ]:
# Read in PIPS data
PIPS_data_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/'
PIPS_filename = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s.nc'.format(date)
PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
PIPS_ds = xr.open_dataset(PIPS_filepath)
print(PIPS_ds)

In [ ]:
PIPS_ds.load()
PIPS_ds_1900_2200 = PIPS_ds.sel(time=slice(starttimestring2, endtimestring2))
hour_min_start_string = datetime_start.strftime('%H%M')
hour_min_end_string = datetime_end.strftime('%H%M')
PIPS_new_file = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s_{}-{}.nc'.format(date, hour_min_start_string,
                                                                              hour_min_end_string)
PIPS_new_filepath = os.path.join(PIPS_data_dir, PIPS_new_file)
PIPS_ds.to_netcdf(PIPS_new_filepath)

In [ ]:
print(PIPS_ds['KHTX_beam_height_at_PIPS'])

In [ ]:
if not already_merged:
    fmcw_merged_dataset = fmcw.correct_fmcw_with_nexrad(fmcw_merged_dataset, PIPS_ds_1900_2200, radar_name='KHTX', 
                                                        dBZ_field='REF', plot=True)
    print(fmcw_merged_dataset['Ze'])
    print(fmcw_merged_dataset['Ze_corr'])
    fmcw.plot_fmcw_4panel_xarray(fmcw_merged_dataset, dBZ_var='Ze_corr')
    print(fmcw_merged_dataset)
    fmcw_filename = 'fmcw_merged_corrected_{}_{}.nc'.format(starttimestring, endtimestring)
    fmcw_filepath = os.path.join(fmcw_output_dir, fmcw_filename)
    fmcw_merged_dataset.to_netcdf(fmcw_filepath)

In [ ]:
%matplotlib inline
# plt.rcParams["figure.figsize"] = (20,10)
# fmcw.plot_fmcw_4panel_xarray(fmcw_merged_dataset)
fmcw_merged_resampled_ds = fmcw_merged_dataset.resample(time='10S').interpolate()
print(fmcw_merged_resampled_ds)
fmcw_merged_ds_rolling = fmcw_merged_resampled_ds.rolling(time=6)
fmcw_merged_resampled_ds = fmcw_merged_ds_rolling.mean()
print(fmcw_merged_resampled_ds)

In [ ]:
fmcw_merged_resampled_at_PIPS_ds = fmcw_merged_resampled_ds.interp_like(PIPS_ds_1900_2200)

In [ ]:
fmcw_merged_resampled_at_PIPS_ds.attrs = fmcw_merged_dataset.attrs
print(fmcw_merged_resampled_at_PIPS_ds)
fmcw.plot_fmcw_4panel_xarray(fmcw_merged_resampled_at_PIPS_ds)

In [ ]:
import pyPIPS.parsivel_params as pp
import pyPIPS.polarimetric as dp
min_diameter = pp.parsivel_parameters['min_diameter_bins_mm']
max_diameter = pp.parsivel_parameters['max_diameter_bins_mm']
bin_width = max_diameter - min_diameter
avg_diameter = pp.parsivel_parameters['avg_diameter_bins_mm']
min_fall_bins = pp.parsivel_parameters['min_fallspeed_bins_mps']
max_fall_bins = pp.parsivel_parameters['max_fallspeed_bins_mps']
avg_fall_bins = pp.parsivel_parameters['avg_fallspeed_bins_mps']

vd_matrix_da = PIPS_ds_1900_2200['VD_matrix_qc']
ND = PIPS_ds_1900_2200['ND_qc']
logND = np.log10(ND)

# conv_df = pips.calc_thermo(conv_df)
# # empirical_fallspeed = pips.calc_empirical_fallspeed(avg_diameter)
# fallspeed_spectrum = pips.calc_fallspeed_spectrum(avg_diameter, avg_fall_bins, correct_rho=True,
#                                                   rho=conv_df['rho'])
# vd_matrix_da = vd_matrix_da.where(vd_matrix_da > 0.0)
# ND = pips.calc_ND(vd_matrix_da, fallspeed_spectrum, 10.)
# ND = ND.T
# logND = np.log10(ND)
# logND = 

PSD_datetimes = pips.get_PSD_datetimes(vd_matrix_da)
PSD_datetimes_dict = pips.get_PSD_time_bins(PSD_datetimes)

PSD_edgetimes = md.date2num(PSD_datetimes_dict['PSD_datetimes_edges'])
PSD_centertimes = md.date2num(PSD_datetimes_dict['PSD_datetimes_centers'])

# ND = PIPS_dict['ND'].T
# logND = ma.log10(ND)
# DSD_index = PIPS_dict['DSD_index']
# DSD_interval = PIPS_dict['DSD_interval']
# DSD_interval_td = timedelta(seconds=DSD_interval)
# DSD_halfinterval_td = timedelta(seconds=DSD_interval / 2.)

# Determine start and end times/indices for analysis
# PSDtimestamps = PIPS_dict['PSDtimestamps']
# PSDtimestampsnums = md.date2num(PSDtimestamps)

#     plotstarttime = starttime
#     plotstoptime = stoptime

# pstartindex, pstopindex = getTimeWindow(starttimestring, endtimestring, PSDtimestampsnums)

# pstarttime = PSDtimestampsnums[pstartindex]
# pstoptime = PSDtimestampsnums[pstopindex]

# plotstarttime = pstarttime
# plotstoptime = pstoptime

# PSDtimestamps_edge = [x - DSD_interval_td for x in PSDtimestamps]
# # Add an extra 10 sec for the last time bin boundary
# PSDtimestamps_edge.append(PSDtimestamps_edge[-1] + DSD_interval_td)
# PSDtimestamps_avg = [x - DSD_halfinterval_td for x in PSDtimestamps]
# PSDstarttimes = md.date2num(PSDtimestamps_edge[pstartindex:pstopindex + 1])
# PSDmidtimes = md.date2num(PSDtimestamps_avg[pstartindex:pstopindex + 1])

scattfile = '/Users/dawson29/Projects/pyPIPS/tmatrix/FMCW/SCTT_RAIN_fw100.dat'
# Observed DSD: TODO, need to make ND, ND_expDSD, and ND_gamDSD have consistent units!
dualpol_dis = dp.calpolrain(15.0, scattfile, ND, bin_width)
#print(dualpol_dis)
dis_ref = dualpol_dis.get('REF', np.empty((0)))
logND_plot = logND.T
disvars = {'min_diameter': min_diameter, 'PSDstarttimes': PSD_edgetimes,
           'PSDmidtimes': PSD_centertimes, 'logND': logND_plot}
disvars['REF'] = dis_ref

# Quick and dirty rayleigh scattering approximation

M6 = []
print(ND.shape)
for t in range(ND.shape[0]):
    temp_M6 = ((avg_diameter[:] / 1000.)**6.) * (1000. * ND[t, :]) * bin_width[:] / 1000.
    temp_M6 = np.nansum(temp_M6)
    M6.append(temp_M6)

M6 = np.array(M6, dtype=np.float64)
refl_disd = 10.0 * np.log10(1e18 * M6)
#disvars['dBZ'] = refl_disd

# fig, ax = plt.subplots(figsize=(8, 3))

# ax.plot(PSD_centertimes, disvars['dBZ'])

radz = 500. # (m) Height of FMCW gate to plot
radzindex = int(radz/5.)

plotx_rad = fmcw_merged_dataset['time'].values
#dBZ_D_plt = fmcw_merged_dataset['Ze'].values[:, radzindex] + 10.**(2.*7.3/10.)
dBZ_D_plt = fmcw_merged_dataset['Ze_corr'].values[:, radzindex]


# print(fmcw_merged_dataset['height'][radzindex])
# print(fmcw_merged_dataset['Ze_corr'][:, radzindex])

# fmcw_merged_dataset['Ze'][:, radzindex].plot()

# print(dBZ_D_plt)

radvars = {'radmidtimes': plotx_rad, 'REF': dBZ_D_plt}

# Prepare axis parameters
# timelimits = [md.date2num(datetime.strptime(starttimestring, tm.timefmt3)),
#               md.date2num(datetime.strptime(endtimestring, tm.timefmt3))]
# timelimits = [md.date2num(datetime.strptime('20170430203000', tm.timefmt3)),
#               md.date2num(datetime.strptime('20170430210000', tm.timefmt3))]
# Set up axis parameters
start_time = '20170327200000'
end_time = '20170327204500'
try:
    start_datetime = datetime.strptime(start_time, tm.timefmt3)
    print('start_datetime', start_datetime)
except (ValueError, TypeError):
    start_datetime = PSD_edgetimes[0]
try:
    end_datetime = datetime.strptime(end_time, tm.timefmt3)
    print('end_datetime', end_datetime)
except (ValueError, TypeError):
    end_datetime = PSD_edgetimes[-1]
timelimits = [start_datetime, end_datetime]
diamlimits = [0.0, 9.0]
diamytick = 1.0

locator = md.MinuteLocator(byminute=np.arange(0., 65., 5.))# md.HourLocator(interval=1)
minorlocator = md.MinuteLocator(byminute=np.arange(0., 61., 1.))
# minorlocator = md.HourLocator(interval=1)
#dateformat = '%d/%H'
dateformat = '%H:%M'
formatter = md.DateFormatter(dateformat)
axparams = {'majorxlocator': locator, 'majorxformatter': formatter,
            'minorxlocator': minorlocator,
            'axeslimits': [timelimits, diamlimits],
            'majorylocator': ticker.MultipleLocator(base=diamytick),
            'axeslabels': [None, 'D (mm)']}

            # Ok, now we should have everything ready to go to plot the meteograms.
            # Let'er rip!
meteogram_image_dir = './'
dis_plot_name = 'PIPS2A_observed'
pm.plotDSDmeteograms(dis_plot_name, meteogram_image_dir,
                     axparams, disvars, radvars.copy(), close_fig=False)

In [ ]:
dBZ_D_plt = fmcw_merged_resampled_ds['Ze_corr']
dBZ_D_plt.sel(height=[500., 750., 1000., 1250., 1500.]).plot.line(x='time')

In [ ]:
fmcw_merged_resampled_at_PIPS_ds_2000_2045 = fmcw_merged_resampled_at_PIPS_ds.sel(
    time=slice('2017-03-27T20:00', '2017-03-27T20:45'))

print(fmcw_merged_resampled_at_PIPS_ds_2000_2045)

fig, ax = plt.subplots(figsize=(10, 10))

# Reflectivity (Ze)
#     m1 = ax[0, 0].pcolormesh(UTCtime, height/1000., Ze.T,cmap=plasma_white, \
#                              vmin=-30., vmax=30.)

m1 = fmcw_merged_resampled_at_PIPS_ds_2000_2045['Ze_corr'].plot(x='time', y='height', ax=ax, 
                                                                cmap=fmcw.plasma_white, vmin=-20., vmax=60.)

ax.set_title("(a) " + fmcw_merged_resampled_at_PIPS_ds_2000_2045['Zef_corr'].name + " (dBZ)")
ax.set_ylim(fmcw_merged_resampled_at_PIPS_ds_2000_2045['height'].min(), 
            fmcw_merged_resampled_at_PIPS_ds_2000_2045['height'].max())
# ax[0, 0].xaxis.set_major_locator(md.MinuteLocator(byminute=range(0, 60, 10)))
# ax[0, 0].xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(mtokm))
#plt.xticks(rotation=30)
#ax[0, 0].set_xlabel("Time (UTC)")
ax.set_ylabel("Height (km AGL)")
# plt.colorbar(m1,ax=ax[0, 0])

#     # Dealiased Doppler (vertical) velocity (vel_da)

#     m2 = dataset['vel_da'].plot(x='time', y='height', ax=ax[0, 1], cmap=cm.RdBu_r,
#                                 vmin=-2.*np.ceil(dataset.Vmax), vmax=2.*np.ceil(dataset.Vmax))

# #     m2 = ax[0, 1].pcolormesh(UTCtime,height/1000., vel.T, cmap=cm.RdBu_r, \
# #                              vmin=-1.*np.ceil(Vmax), vmax=np.ceil(Vmax))

#     ax[0, 1].set_title("(b) " + dataset['vel_da'].name + r" (m s$^{-1}$)")
#     ax[0, 1].set_ylim(dataset['height'].min(), dataset['height'].max())
#     # ax[0, 1].xaxis.set_major_locator(md.MinuteLocator(byminute=range(0, 60, 10)))
#     # ax[0, 1].xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
#     ax[0, 1].yaxis.set_major_formatter(ticker.FuncFormatter(mtokm))
#     #plt.xticks(rotation=30)
#     #ax[0, 1].set_xlabel("Time (UTC)")
#     #ax[0, 1].set_ylabel("Height (km AGL)")
#     # plt.colorbar(m2,ax=ax[0, 1])

In [ ]:
vbins = fmcw_merged_resampled_at_PIPS_ds['vels']
vbins_dealias = vbins - vbins[-1]
fmcw_merged_resampled_at_PIPS_ds.coords['vels_dealiased'] = ('vels', vbins_dealias)
#print(fmcw_merged_resampled_at_PIPS_ds)
Zef_corr = fmcw_merged_resampled_at_PIPS_ds['Zef_corr']
#print(Zef_corr)
#print(Zef_corr.loc['2017-04-30T22:30':'2017-04-30T22:31'])
Zef_corr_onetime = Zef_corr.sel(time='2017-03-27T20:12')
# print(Zef_corr_2230)
Zef_corr_onetime = Zef_corr_onetime.roll(vels=128, roll_coords=False)
#delta_v = Zef_corr_2230.vels[1] - Zef_corr_2230.vels[0]
#print(2. / delta_v)
#shift = int(2. / delta_v)
#Zef_corr_2230_shifted = Zef_corr_2230.roll(vels=shift, roll_coords=False)
Zef_corr_onetime_z2000 = Zef_corr_onetime.sel(height=500.)
Zef_corr_onetime.plot(x='vels_dealiased', y='height', vmin=-20, vmax=60)
#fig2, ax2 = plt.subplots()
#Zef_corr_2230_shifted.plot(ax=ax2, vmin=-30, vmax=30)
fig, ax = plt.subplots()
Zef_corr_onetime_z2000.plot(ax=ax, x='vels_dealiased')

#Zef_corr_2230.plot(vmin=0., vmax=40.)

In [ ]:
# Let's make a ridge plot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Create the data
rs = np.random.RandomState(1979)
x = rs.randn(500)
g = np.tile(list("ABCDEFGHIJ"), 50)
df = pd.DataFrame(dict(x=x, g=g))
m = df.g.map(ord)
df["x"] += m

print(df)


Zef_corr_2230_thinned = Zef_corr_2230.thin({'height': 100})

Zef_corr_2230_df = Zef_corr_2230_thinned.to_dataframe().reset_index()
print(Zef_corr_2230_df)
# Initialize the FacetGrid object
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(Zef_corr_2230_df, row="height", hue='height', aspect=2, height=1., palette=pal)

# Draw the densities in a few steps
#g.map(sns.kdeplot, "vels_dealiased", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
#g.map(sns.kdeplot, "vels_dealiased", clip_on=False, color="w", lw=2, bw=.2)
g.map(sns.lineplot, "vels_dealiased", "Zef_corr", clip_on=False, alpha=1, lw=1.5)
#g.map(sns.lineplot, "vels_dealiased", "Zef_corr", clip_on=False, color="w", lw=2, bw=.2)
#g.map(plt.axhline, y=0, lw=2, clip_on=False)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)


g.map(label, "vels_dealiased")

# Set the subplots to overlap
# g.fig.subplots_adjust(hspace=-.25)

# Remove axes details that don't play well with overlap
g.set_titles("")
#g.set(yticks=[])
#g.despine(bottom=True, left=True)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Create the data
rs = np.random.RandomState(1979)
x = rs.randn(500)
g = np.tile(list("ABCDEFGHIJ"), 50)
df = pd.DataFrame(dict(x=x, g=g))
m = df.g.map(ord)
df["x"] += m
print(df)

# Initialize the FacetGrid object
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(df, row="g", hue="g", aspect=15, height=.5, palette=pal)

# Draw the densities in a few steps
g.map(sns.kdeplot, "x", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
g.map(sns.kdeplot, "x", clip_on=False, color="w", lw=2, bw=.2)
g.map(plt.axhline, y=0, lw=2, clip_on=False)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)


g.map(label, "x")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-.25)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)